In [1]:
from datasets import load_dataset
import numpy as np
from datasets import Dataset
import ast
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer
from datasets import load_metric
from transformers import DataCollatorForTokenClassification

# Import Data

In [2]:
# какая-то ошибка в ner список на самом деле в формате string
tr = load_dataset('csv', data_files='data/ner/trainfold_0.csv').remove_columns('Unnamed: 0')
tr = tr.shuffle(seed= 14)
ts = load_dataset('csv', data_files='data/ner/testfold_0.csv').remove_columns('Unnamed: 0')
ts = ts.shuffle(seed= 14)
ts

Found cached dataset csv (C:/Users/intern-lab/.cache/huggingface/datasets/csv/default-6ceff85eb6c91486/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at C:\Users\intern-lab\.cache\huggingface\datasets\csv\default-6ceff85eb6c91486\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-4e7348e6d444e196.arrow
Found cached dataset csv (C:/Users/intern-lab/.cache/huggingface/datasets/csv/default-878044040cc39661/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at C:\Users\intern-lab\.cache\huggingface\datasets\csv\default-878044040cc39661\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-be471511e3649998.arrow


DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'ner', 'task'],
        num_rows: 157
    })
})

In [4]:
# соотношения индексов к тегу
id2label = {0 : 'O', 1 : 'PAT', 2 : 'PHY', 3 : 'PAT-START', 4 : 'PHY-START'}
label2id = {'O' : 0, 'PAT' : 1, 'PHY' : 2, 'PAT-START' : 3, 'PHY-START' : 4}
label_list = ['O', 'PAT', 'PHY', 'PAT-START', 'PHY-START']

In [5]:
# функция для преобразования из str в list
def from_str_to_list(data):
    out = []
    for i in data:
        out.append(ast.literal_eval(i))
    return out

In [6]:
# преобразование в list
train = tr['train']
test = ts['train']

trner = train['ner']
trwords = train['words']

tsner = test['ner']
tswords = test['words']

In [7]:
# преобразование в list
train = train.remove_columns('ner')
train = train.add_column('ner',from_str_to_list(trner))
train = train.remove_columns('words')
train = train.add_column('words',from_str_to_list(trwords))

test = test.remove_columns('ner')
test = test.add_column('ner',from_str_to_list(tsner))
test = test.remove_columns('words')
test = test.add_column('words',from_str_to_list(tswords))

Loading cached processed dataset at C:\Users\intern-lab\.cache\huggingface\datasets\csv\default-6ceff85eb6c91486\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-9842681c3e77d2a5.arrow
Loading cached processed dataset at C:\Users\intern-lab\.cache\huggingface\datasets\csv\default-878044040cc39661\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-687d16dc0cf5f9a5.arrow


# Load Tokenizer

In [15]:
# функция токенизации
max_length = 512
def tokenize_and_align_labels(examples): # trunc texts and labels
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["words"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(label2id[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [16]:
# применение токенизации
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenized_train = train.map(tokenize_and_align_labels, batched=True)
tokenized_test = test.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/628 [00:00<?, ? examples/s]

Map:   0%|          | 0/157 [00:00<?, ? examples/s]

# Model

In [14]:
# функция подсчета метрики
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}

C:\Users\intern-lab\AppData\Local\Temp\ipykernel_11152\339056260.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [15]:
# параметры модели
model = AutoModelForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3,
                                                        label2id=label2id,
                                                        id2label = id2label,
                                                        ignore_mismatched_sizes=True)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [16]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [17]:
# аргументы обучения модели
task = "ner" 
args = TrainingArguments(
    output_dir = 'models',
    evaluation_strategy = "epoch",
    learning_rate = 1e-4,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    num_train_epochs = 2,
    weight_decay = 1e-5,
)

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_test,
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

In [ ]:
# старт тренировки и финальная валидация
trainer.train()
trainer.evaluate()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: words, task, id, ner. If words, task, id, ner are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
C:\Users\intern-lab\AppData\Local\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 628
  Num Epochs = 2
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 628
  Number of trainable parameters = 177265155
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `